**GO Relationship Import**

First, get the "is_a" relationship.

In [4]:
import re
import json

# Open the input file for reading
with open('go_2024-07-17.obo', 'r') as input_file:
    # Open a file for writing the tab-delimited rows
    with open('is_a_temp.tsv', 'w') as output_file:
        # Write header
        output_file.write("id\tid2\n")

        # Initialize variables to store id and relationships
        current_id = ""
        is_a = []

        # Iterate through each line in the input file
        for line in input_file:
            line = line.strip()

            # Check if the line starts with "[Term]"
            if line.startswith("[Term]"):
                # If a previous term had relationships, write them to the output file
                if current_id and is_a:
                    for i in is_a:
                        output_file.write(f"{current_id}\t{i}\n")

                # Reset variables for the new term
                current_id = ""
                is_a = []
            elif line.startswith("id:"):
                current_id = line.replace("id: ", "")
            elif line.startswith("is_a:"):
                is_a.append(line.split("is_a:")[1].strip().split()[0])

        # Write the last term's relationships if any
        if current_id and is_a:
            for i in is_a:
                output_file.write(f"{current_id}\t{i}\n")

Next add a new column to ease import into Neo4j.

In [5]:
import pandas as pd
import os

# Step 2: Read the tab-delimited file into a DataFrame
file_path = 'is_a_temp.tsv'
df = pd.read_csv(file_path, sep='\t')

# Step 3: Add a new column to the DataFrame
df['is_a'] = 'is_a'

# Step 4: Save the modified DataFrame to a new file
output_file_path = 'is_a_import_2024-07-17.tsv'
df.to_csv(output_file_path, sep='\t', index=False)
os.remove(file_path)

# Display the DataFrame
print(df)

                      id          id2  is_a
0             GO:0000001   GO:0048308  is_a
1             GO:0000001   GO:0048311  is_a
2             GO:0000002   GO:0007005  is_a
3             GO:0000006   GO:0005385  is_a
4             GO:0000007   GO:0005385  is_a
...                  ...          ...   ...
66170  term_tracker_item   GO:0120255  is_a
66171  term_tracker_item   GO:2001316  is_a
66172  term_tracker_item  ends_during  is_a
66173  term_tracker_item    regulates  is_a
66174  term_tracker_item    regulates  is_a

[66175 rows x 3 columns]


Delete the last 8 lines that start with "term_tracker_item".

Now extract the relationship data between GO terms.

In [2]:
# Open the input file for reading
with open('go_2024-07-17.obo', 'r') as input_file:
    # Open a file for writing the tab-delimited rows
    with open('go-relationship_temp.tsv', 'w') as output_file:
        # Write header
        output_file.write("id\trelationship\n")

        # Initialize variables to store id and relationships
        current_id = ""
        relationships = []

        # Iterate through each line in the input file
        for line in input_file:
            line = line.strip()

            # Check if the line starts with "[Term]"
            if line.startswith("[Term]"):
                # If a previous term had relationships, write them to the output file
                if current_id and relationships:
                    for relationship in relationships:
                        output_file.write(f"{current_id}\t{relationship}\n")

                # Reset variables for the new term
                current_id = ""
                relationships = []
            elif line.startswith("id:"):
                current_id = line.replace("id: ", "")
            elif line.startswith("relationship:"):
                relationships.append(line.split("relationship:")[1].strip())

        # Write the last term's relationships if any
        if current_id and relationships:
            for relationship in relationships:
                output_file.write(f"{current_id}\t{relationship}\n")


Parse only the relevant relationship data from the new tab delimited file.

In [3]:
import pandas as pd
import os

file_path = 'go-relationship_temp.tsv'
df = pd.read_csv(file_path, sep='\t')

df['id2'] = df['relationship'].str.split(' ').str[1]

df['relationship'] = df['relationship'].str.split(' ').str[0]

output_file_path = 'relationship_import_2024-07-17.tsv'
df.to_csv(output_file_path, sep='\t', index=False)
os.remove('go-relationship_temp.tsv')

# Display the DataFrame
print(df)

               id relationship         id2
0      GO:0000015      part_of  GO:0005829
1      GO:0000018    regulates  GO:0006310
2      GO:0000019    regulates  GO:0006312
3      GO:0000022      part_of  GO:0000070
4      GO:0000022      part_of  GO:0007052
...           ...          ...         ...
17590  GO:2001310     has_part  GO:0003839
17591  GO:2001310     has_part  GO:0004364
17592  GO:2001310     has_part  GO:0004497
17593  GO:2001310     has_part  GO:0008170
17594  GO:2001310     has_part  GO:0008483

[17595 rows x 3 columns]
